In [6]:
import json
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import seaborn
import tidytcells

seaborn.set_theme()
seaborn.set_style("darkgrid")
plt.tight_layout()

<Figure size 640x480 with 0 Axes>

In [7]:
raw_dir = Path(
    "/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/raw/tanno"
)
preprocessed_dir = Path(
    "/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/tanno"
)

In [8]:
example = pd.read_csv(raw_dir / "data" / "A1 memory.txt", sep="\t")

In [9]:
example.sort_values(by="Clustered", ascending=False).head()

,Clustered,Collapsed,ClusterID,CDRH3_AA,CDRL3_AA,CDRH3_NT,CDRL3_NT,VH,DH,JH,VL,JL
0,8899,5544,1,CASSLQGARETQYF,CALLSTGGGNKLTF,TGTGCCAGCAGCTTACAGGGGGCGCGGGAGACCCAGTACTTC,TGTGCTCTCCTTTCCACGGGAGGAGGAAACAAACTCACCTTT,TRBV11-2,TRBD1,TRBJ2-5,TRAV9-2,TRAJ10
1,1945,1077,2,CSWKREIVEQFF,CAVSDRNTNAGKSTF,TGCAGTTGGAAGCGGGAGATAGTTGAGCAGTTCTTC,TGTGCTGTGAGTGATAGGAACACCAATGCAGGCAAATCAACCTTT,TRBV20-1,TRBD2,TRBJ2-1,TRAV8-4,TRAJ27
2,1215,867,3,CASSSQGANTEAFF,CIVRKRNQFYF,TGTGCCAGCAGCTCCCAAGGAGCGAACACTGAAGCTTTCTTT,TGCATCGTCAGAAAACGTAACCAGTTCTATTTT,TRBV9,NaN,TRBJ1-1,TRAV26-1,TRAJ49
3,1185,866,4,CASSFQGQGGQPQHF,CAVRPRDTGGFKTIF,TGTGCCAGCAGTTTCCAGGGACAGGGCGGTCAGCCCCAGCATTTT,TGTGCTGTGAGGCCTCGGGATACTGGAGGCTTCAAAACTATCTTT,TRBV12-3,TRBD1,TRBJ1-5,TRAV21,TRAJ9
4,1002,785,5,CSAPLAGVSYNEQFF,CIVRSYNYGQNFVF,TGCAGTGCGCCACTAGCGGGCGTCTCCTACAATGAGCAGTTCTTC,TGCATCGTCAGATCTTATAACTATGGTCAGAATTTTGTCTTT,TRBV20-1,TRBD2,TRBJ2-1,TRAV26-1,TRAJ26


In [21]:
dfs = []

for f in (raw_dir / "data").iterdir():
    df = pd.read_csv(f, sep="\t")

    # Filter for potential mispairings
    df = df.sort_values(by="Clustered", ascending=False)
    df = df.drop_duplicates("CDRH3_NT", keep="first")
    df = df.drop_duplicates("CDRL3_NT", keep="first")

    df = df[["VL", "CDRL3_AA", "JL", "VH", "CDRH3_AA", "JH"]]
    df.columns = ["TRAV", "CDR3A", "TRAJ", "TRBV", "CDR3B", "TRBJ"]

    dfs.append(df)

combined = pd.concat(dfs, axis="index")

combined["clone_count"] = 1
combined = combined.groupby(
    ["TRAV", "CDR3A", "TRAJ", "TRBV", "CDR3B", "TRBJ"],
    as_index=False,
    dropna=False
).aggregate({"clone_count": "sum"})
combined[["Epitope", "MHCA", "MHCB"]] = pd.NA 
combined = combined[["TRAV", "CDR3A", "TRAJ", "TRBV", "CDR3B", "TRBJ", "Epitope", "MHCA", "MHCB", "clone_count"]]
combined = combined.sort_values(by=combined.columns.tolist(), ignore_index=True)

# Standise
combined["TRAV"] = combined["TRAV"].map(
    lambda x: tidytcells.tcr.standardise(x, enforce_functional=True)
)
combined["TRAJ"] = combined["TRAJ"].map(
    lambda x: tidytcells.tcr.standardise(x, enforce_functional=True)
)
combined["TRBV"] = combined["TRBV"].map(
    lambda x: tidytcells.tcr.standardise(x, enforce_functional=True)
)
combined["TRBJ"] = combined["TRBJ"].map(
    lambda x: tidytcells.tcr.standardise(x, enforce_functional=True)
)

/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV11" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV11".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV15" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV15".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV32" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV32".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV33" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV33".
  warn(
/home/yutanagano/Projects/tcr_em

In [22]:
combined[combined["clone_count"] > 1]

,TRAV,CDR3A,TRAJ,TRBV,CDR3B,TRBJ,Epitope,MHCA,MHCB,clone_count
5,TRAV1-1,CAAASGFQKLVF,TRAJ8,TRBV20-1,CSGRYDPNPEQYF,TRBJ2-7,NaN,NaN,NaN,2
20,TRAV1-1,CAAFRGQNFVF,TRAJ26,TRBV4-1,CASSQVTGGNSYEQYF,TRBJ2-7,NaN,NaN,NaN,2
26,TRAV1-1,CAAGGSNYQLIW,TRAJ33,TRBV12-3,CASSSSGANTQYF,TRBJ2-3,NaN,NaN,NaN,2
28,TRAV1-1,CAAGGYNKLIF,TRAJ4,TRBV7-2,CASSLKTGSVNEQFF,TRBJ2-1,NaN,NaN,NaN,2
32,TRAV1-1,CAAGSSASKIIF,TRAJ3,TRBV4-2,CASSQAQSNYGYTS,TRBJ1-2,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...
817977,TRAV9-2,CASRDDKIIF,TRAJ30,TRBV7-6,CASSRYNEKLFF,TRBJ1-4,NaN,NaN,NaN,2
818000,TRAV9-2,CASRTGFQKLVF,TRAJ8,TRBV30,CAWRDTRNYGYTF,TRBJ1-2,NaN,NaN,NaN,2
818002,TRAV9-2,CASRTRSGGSNYKLTF,TRAJ53,TRBV20-1,CSAGQGGGSPLHF,TRBJ1-6,NaN,NaN,NaN,2
818149,TRAV9-2,CAVSNTGKLIF,TRAJ37,TRBV7-8,CASSPLPGSGETQYF,TRBJ2-5,NaN,NaN,NaN,2


In [24]:
bv_counts = dict()

for bv in combined["TRBV"].dropna().unique():
    bv_count = combined["clone_count"][combined["TRBV"] == bv].sum()
    bv_counts[bv] = int(bv_count)

In [25]:
with open("trbv_frequencies.json", "w") as f:
    json.dump(bv_counts, f, indent=4)

In [26]:
combined.to_csv(preprocessed_dir / "combined.csv", index=False)

In [27]:
test_frac = 0.1
test_rows = int(test_frac * len(combined))

shuffled = combined.sample(frac=1, random_state=420)

test = shuffled.iloc[:test_rows]
train = shuffled.iloc[test_rows:]

In [28]:
travs = tidytcells.tcr.query(precision="gene", functionality="F", contains="TRAV")
trajs = tidytcells.tcr.query(precision="gene", functionality="F", contains="TRAJ")
trbvs = tidytcells.tcr.query(precision="gene", functionality="F", contains="TRBV")
trbjs = tidytcells.tcr.query(precision="gene", functionality="F", contains="TRBJ")

In [29]:
for dataset in (train, test):
    assert dataset["TRAV"].nunique() == len(travs)
    assert not (set(travs) - set(dataset["TRAV"]))
    assert dataset["TRAJ"].nunique() == len(trajs)
    assert not (set(trajs) - set(dataset["TRAJ"]))
    assert dataset["TRBV"].nunique() == len(trbvs)
    assert not (set(trbvs) - set(dataset["TRBV"]))
    assert dataset["TRBJ"].nunique() == len(trbjs)
    assert not (set(trbjs) - set(dataset["TRBJ"]))
    print(dataset["TRBV"].value_counts())

TRBV
TRBV20-1    97492
TRBV12-3    57756
TRBV7-2     51351
TRBV30      38143
TRBV5-1     36462
TRBV7-9     33495
TRBV9       27154
TRBV27      26669
TRBV4-1     26585
TRBV2       24967
TRBV6-5     22318
TRBV29-1    19902
TRBV4-2     17603
TRBV19      17273
TRBV28      16807
TRBV6-2     16661
TRBV18      15992
TRBV6-1     15623
TRBV10-3    13384
TRBV4-3     13211
TRBV3-1     13018
TRBV11-2    12839
TRBV15      12186
TRBV5-4     11898
TRBV25-1    11284
TRBV13      11080
TRBV7-6     10278
TRBV7-3      9948
TRBV6-6      9174
TRBV14       8857
TRBV7-8      8373
TRBV24-1     5257
TRBV11-3     4172
TRBV11-1     3630
TRBV7-7      3599
TRBV12-5     2601
TRBV10-2     2230
TRBV5-8      1975
TRBV6-4      1806
TRBV10-1      950
TRBV16        506
TRBV7-4       389
TRBV12-4      150
TRBV6-9       144
TRBV6-8       141
TRBV6-3        99
TRBV5-6        55
TRBV5-5        52
Name: count, dtype: int64
TRBV
TRBV20-1    10904
TRBV12-3     6441
TRBV7-2      5704
TRBV30       4296
TRBV5-1      4077
TRBV7-9   

In [30]:
train.to_csv(preprocessed_dir / "train.csv", index=False)
test.to_csv(preprocessed_dir / "test.csv", index=False)

In [31]:
test.sample(n=5, random_state=420).to_csv(
    preprocessed_dir / "exemplars.csv", index=False
)